# Robustness of policies

## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ema_workbench.analysis import parcoords

from problem_formulation import get_model_for_problem_formulation
dike_model, planning_steps = get_model_for_problem_formulation(3)

C:\Users\thori\Anaconda3\envs\mbdm1\lib\site-packages\ema_workbench\analysis\prim.py:30: ImportWarning: altair based interactive inspection not available
  warnings.warn(("altair based interactive "


The results from the optimisation are read in. Only policies i.e. lever combinations are kept, that result in less than 0.01 expected deaths for A.5 Deventer. This ensures that all lever combinations that are considered for policies actually perform well over the worst case scenarios.

In [2]:
results = []
for i in range(5):
    results_scenario = pd.read_csv(f'outcomes/15k_results_optimisation_scenario{i}.csv')
    results_scenario = results_scenario[results_scenario['A.5_Expected Number of Deaths'] <= 0.01]
    results.append(results_scenario)

In [3]:
len(results)

5

In [4]:
results[3].columns

Index(['Unnamed: 0', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1',
       '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1',
       '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs',
       'A.2_Expected Number of Deaths', 'A.3 Total Costs',
       'A.3_Expected Number of Deaths', 'A.4 Total Costs',
       'A.4_Expected Number of Deaths', 'A.5 Total Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

Following, the results for the 4 other dike rings (non-Deventer) are added up. This is due to the comparison between Deventer and the rest. Results for dike ring 4 (Gorssel) are kept to be able to directly compare Deventer's results to those of its neighbour Gorssel.

In [5]:
for i in range(len(results)):
    results[i]['Other Areas Total Number of Deaths'] = 0  
    results[i]['Other Areas Total Costs'] = 0
    results[i]['Scenario number'] = i
    
    for area in range(1,5):
        results[i]['Other Areas Total Number of Deaths'] += results[i][f'A.{area}_Expected Number of Deaths']
        results[i]['Other Areas Total Costs'] += results[i][f'A.{area} Total Costs']
    
    results[i] = results[i].drop(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs',
       'A.2_Expected Number of Deaths', 'A.3 Total Costs',
       'A.3_Expected Number of Deaths', 'Unnamed: 0'], axis=1)

In [6]:
#combined_results = results[0].append([results[1], results[2], results[3], results[4]], ignore_index=True)

From the 5 scenarios that were put into the optimisation (see notebook multi-scenario MORDM.ipynb), the maximum i.e. worst values for each of those scenarios for each column as well as the minimum i.e. best outcomes for those 5 optimised scenarios are taken. Then, the experiments matching those outcomes are are stored. 

In [7]:
outcomes_array = ['A.4 Total Costs', 
                  'A.4_Expected Number of Deaths',
                  'A.5 Total Costs',
                  'A.5_Expected Number of Deaths', 
                  'RfR Total Costs',
                  'Expected Evacuation Costs',
                  'Other Areas Total Number of Deaths',
                  'Other Areas Total Costs']


lever_array = outcomes_array + ['Scenario number']

result_best_worst_scenarios = []
for i in range(len(results)):
    result_best_worst = []
    result_worst = results[i].loc[results[i][outcomes_array].idxmax()]

    result_best = results[i].loc[results[i][outcomes_array].idxmin()]

    result_best_worst = result_best.append(result_worst)
    
    result_best_worst = result_best_worst.drop(lever_array, axis=1)
    result_best_worst_scenarios.append(result_best_worst)

In [8]:
len(result_best_worst_scenarios)

5

In [9]:
result_best_worst_all = result_best_worst_scenarios[0].append([result_best_worst_scenarios[1], result_best_worst_scenarios[2], result_best_worst_scenarios[3], result_best_worst_scenarios[4]], ignore_index=True)
result_best_worst_all

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
0,0,0,1,0,0,0,0,0,1,1,...,3,0,2,0,4,0,0,5,6,2
1,0,0,1,0,0,0,0,0,1,1,...,3,0,0,0,4,0,4,10,1,5
2,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,4,6,6,7,0,0
3,0,0,1,0,0,0,0,0,1,1,...,3,0,0,0,4,0,4,10,1,5
4,0,0,0,0,0,0,0,0,0,0,...,6,3,3,3,7,2,0,9,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,1,0,1,0,0,1,1,0,0,...,0,2,0,0,3,0,2,6,3,3
76,1,1,1,1,0,1,1,1,1,1,...,2,4,1,7,2,0,0,7,3,6
77,0,0,1,0,0,0,0,0,1,0,...,0,1,0,7,0,0,0,6,2,7
78,0,0,1,0,0,0,1,0,0,0,...,2,0,0,1,6,0,0,9,2,2


It results, that some experiments are the same. Duplicates will be removed. Then, out of those experiments policies are created. 

In [10]:
result_best_worst_all.duplicated(subset= None, keep = 'first')

0     False
1     False
2     False
3      True
4     False
      ...  
75    False
76    False
77    False
78    False
79    False
Length: 80, dtype: bool

In [11]:
duplicateDFRow = result_best_worst_all[result_best_worst_all.duplicated()]
result_best_worst_all_no_duplicates = result_best_worst_all.drop(duplicateDFRow.index)

In [12]:
result_best_worst_all_no_duplicates

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
0,0,0,1,0,0,0,0,0,1,1,...,3,0,2,0,4,0,0,5,6,2
1,0,0,1,0,0,0,0,0,1,1,...,3,0,0,0,4,0,4,10,1,5
2,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,4,6,6,7,0,0
4,0,0,0,0,0,0,0,0,0,0,...,6,3,3,3,7,2,0,9,4,2
5,0,1,1,0,0,0,1,0,0,1,...,5,1,6,0,4,2,0,7,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,1,0,1,0,0,1,1,0,0,...,0,2,0,0,3,0,2,6,3,3
76,1,1,1,1,0,1,1,1,1,1,...,2,4,1,7,2,0,0,7,3,6
77,0,0,1,0,0,0,0,0,1,0,...,0,1,0,7,0,0,0,6,2,7
78,0,0,1,0,0,0,1,0,0,0,...,2,0,0,1,6,0,0,9,2,2


In [13]:
from ema_workbench import Policy

policies = []
for j, row in result_best_worst_all_no_duplicates.iterrows():
        policy = Policy(f'scenario option {j}', **row.to_dict())
        policies.append(policy)

These policies represent the policies that perform best and worst over the worst-case scenarios but still meet the set threshold of 0.01 deaths in Deventer. 1000 scenarios are run with these policies to explore each policy's robustness.

In [18]:
from ema_workbench import (MultiprocessingEvaluator)
from ema_workbench.util import ema_logging

ema_logging.log_to_stderr(level = "NOTSET")


with MultiprocessingEvaluator(dike_model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(10, policies=policies)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10 scenarios * 66 policies * 1 model(s) = 660 experiments
[MainProcess/DEBUG] 1 cases completed
[MainProcess/DEBUG] storing A.1 Total Costs
[MainProcess/DEBUG] storing A.1_Expected Number of Deaths
[MainProcess/DEBUG] storing A.2 Total Costs
[MainProcess/DEBUG] storing A.2_Expected Number of Deaths
[MainProcess/DEBUG] storing A.3 Total Costs
[MainProcess/DEBUG] storing A.3_Expected Number of Deaths
[MainProcess/DEBUG] storing A.4 Total Costs
[MainProcess/DEBUG] storing A.4_Expected Number of Deaths
[MainProcess/DEBUG] storing A.5 Total Costs
[MainProcess/DEBUG] storing A.5_Expected Number of Deaths
[MainProcess/DEBUG] storing RfR Total Costs
[MainProcess/DEBUG] storing Expected Evacuation Costs
[MainProcess/DEBUG] 2 cases completed
[MainProcess/DEBUG] storing A.1 Total Costs
[MainProcess/DEBUG] storing A.1_Expected Number of Deaths
[MainProcess/DEBUG] storing A.2 Total Costs
[MainProcess/DEBUG] storing A.2_Expected Number of

In [24]:
experiments = reeevaluation_results[0]
results = pd.DataFrame(reeevaluation_results[1])

In [25]:
results.to_csv('./outcomes/reevaluation_promising_policies_results.csv')
experiments.to_csv('./outcomes/reevaluation_promising_experiments.csv')

In [14]:
results_load = pd.read_csv('./outcomes/reevaluation_promising_policies_results.csv')
experiments_load = pd.read_csv('./outcomes/reevaluation_promising_experiments.csv')

In [15]:
results_load

,Unnamed: 0,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0,1.876591e+08,0.003849,4.978470e+07,0.000337,9.262480e+07,0.020368,9.695195e+06,0.000000,1.242279e+08,0.0,735000000.0,6510.069154
1,1,1.540026e+08,0.000000,3.918009e+08,0.040629,2.216782e+07,0.000000,9.695195e+06,0.000000,1.242279e+08,0.0,735000000.0,26694.874146
2,2,1.540026e+08,0.000000,1.477499e+08,0.010005,2.216782e+07,0.000000,9.695195e+06,0.000000,1.242279e+08,0.0,735000000.0,6081.861067
3,3,1.540026e+08,0.000000,5.584749e+07,0.001365,2.216782e+07,0.000000,9.695195e+06,0.000000,1.242279e+08,0.0,735000000.0,754.377861
4,4,1.540026e+08,0.000000,6.801229e+07,0.002470,4.994227e+07,0.007128,9.695195e+06,0.000000,1.242279e+08,0.0,735000000.0,3198.881920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,655,1.326461e+08,0.000000,2.106806e+08,0.000000,1.196909e+08,0.000000,5.056398e+07,0.004046,1.643440e+08,0.0,115300000.0,810.461459
656,656,1.326461e+08,0.000000,2.106806e+08,0.000000,1.196909e+08,0.000000,1.265331e+08,0.015275,1.643440e+08,0.0,115300000.0,3518.764246
657,657,1.326461e+08,0.000000,2.110890e+08,0.000110,2.900200e+08,0.079100,1.105718e+08,0.010192,1.643440e+08,0.0,115300000.0,5179.928038
658,658,1.326461e+08,0.000000,2.118950e+08,0.000321,1.196909e+08,0.000000,3.621049e+07,0.001399,1.643440e+08,0.0,115300000.0,250.625258


### Maximin
Maximin is taking into account the worst case scenario the worst thing that could happen.

"the maximin metric has a very high level of intrinsic risk aversion, as its calculation is only based on the worst performance over all scenarios considered (Table 3), leading to a very conservative solution (Bertsimas & Sim, 2004). Similarly, the minimax regret metric assumes that the selected decision alternative will have the largest regret possible, as its calculation is based on the worst-case relative performance" https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000649


In [16]:
result_worst_reevaluation = results_load.loc[results_load[['A.5 Total Costs','A.5_Expected Number of Deaths']].idxmax()]

experiment_worst_reevaluation = experiments_load.iloc[result_worst_reevaluation.index].reset_index()

In [17]:
experiment_worst_reevaluation

,index,Unnamed: 0,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,200,200,118.0,240.189839,1.5,0.136262,339.91677,10.0,0.838754,235.411771,...,4.0,4.0,10.0,3.0,4.0,9.0,9.0,0,scenario option 26,dikesnet
1,240,240,118.0,240.189839,1.5,0.136262,339.91677,10.0,0.838754,235.411771,...,4.0,0.0,1.0,2.0,7.0,0.0,0.0,0,scenario option 30,dikesnet


In [18]:
worst_policies = []

for i in range(len(policies)):
    for j in range(len(experiment_worst_reevaluation)):
        if policies[i].name == experiment_worst_reevaluation['policy'][j]:
            worst_policies.append(policies[i])


In [19]:
worst_policies[1].name

'scenario option 30'

In [20]:
worst_policies

[Policy({'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 1, '1_RfR 1': 1, '1_RfR 2': 1, '2_RfR 0': 1, '2_RfR 1': 1, '2_RfR 2': 1, '3_RfR 0': 0, '3_RfR 1': 1, '3_RfR 2': 1, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 1, 'EWS_DaysToThreat': 3, 'A.1_DikeIncrease 0': 6, 'A.1_DikeIncrease 1': 4, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 9, 'A.2_DikeIncrease 1': 1, 'A.2_DikeIncrease 2': 6, 'A.3_DikeIncrease 0': 8, 'A.3_DikeIncrease 1': 2, 'A.3_DikeIncrease 2': 4, 'A.4_DikeIncrease 0': 4, 'A.4_DikeIncrease 1': 10, 'A.4_DikeIncrease 2': 3, 'A.5_DikeIncrease 0': 4, 'A.5_DikeIncrease 1': 9, 'A.5_DikeIncrease 2': 9}),
 Policy({'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 0, '2_RfR 1': 0, '2_RfR 2': 0, '3_RfR 0': 0, '3_RfR 1': 0, '3_RfR 2': 0, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 0, 'EWS_DaysToThreat': 0, 'A.1_DikeIncrease 0': 5, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 1, 'A.2_DikeIncrease 0': 0, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncr

### Minimax Regret Criterion
The Minimax Regret Criterion is a technique used to make decisions under uncertainty. The context of a decision making process under uncertainty, a decision maker is faced to uncertain states of nature and a number of decision alternatives that can be chosen. The decision made and the final state of nature (which the decision maker does not know beforehand) determines the payoff.

Under this Minimax Regret Criterion, the decision maker calculates the maximum opportunity loss values (or also known as regret) for each alternative, and then she chooses the decision that has the lowest maximum regret.

The regret or opportunity loss for a specific alternative, at a given state of nature, is how much we lose by choosing that alternative and not the optimal alternative, given that state of nature (if the current alternative IS the optima alternative, then the opportunity loss for that alternative, given the state of nature, is 0).

https://mathcracker.com/minimax-regret-criterion-calculator